compare calibration result and simulation result in the instant loop detector

In [1]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-02'
detector = 'e2w_out'
number = 0
path  = "data/sim_data/"
if number<1:
    postfix = f"{detector}_{date}"
else:
    postfix = f"{detector}_{date}_{number}"

instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan/notebook
Current directory: /home/kaveh/projects/Hornsgatan


In [2]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [3]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,e2w_out,1.577923e+09,enter,0_e2w_out,14.99,5.00,DEFAULT_VEHTYPE,None
3,e2w_out,1.577923e+09,enter,1_e2w_out,7.08,5.00,DEFAULT_VEHTYPE,18.28
6,e2w_out,1.577923e+09,enter,2_e2w_out,11.66,5.00,DEFAULT_VEHTYPE,66.19
8,e2w_out,1.577923e+09,enter,3_e2w_out,10.41,5.00,DEFAULT_VEHTYPE,14.00
11,e2w_out,1.577923e+09,enter,4_e2w_out,10.83,5.00,DEFAULT_VEHTYPE,39.65
...,...,...,...,...,...,...,...,...
2152,e2w_out,1.578009e+09,enter,801_e2w_out,8.33,5.00,DEFAULT_VEHTYPE,170.09
2154,e2w_out,1.578009e+09,enter,802_e2w_out,12.91,5.00,DEFAULT_VEHTYPE,136.34
2156,e2w_out,1.578009e+09,enter,803_e2w_out,12.50,5.00,DEFAULT_VEHTYPE,311.22
2159,e2w_out,1.578010e+09,enter,804_e2w_out,5.41,5.00,DEFAULT_VEHTYPE,153.33


In [4]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_e2w_out,1.577923e+09,14.994000,1.80,1577923299,1577923283,max,15.000000,-0.24,-0.006000
1,1_e2w_out,1.577923e+09,7.080500,0.85,1577923317,1577923284,max,6.944444,0.37,0.136056
2,2_e2w_out,1.577923e+09,11.661995,1.40,1577923384,1577923364,max,11.388889,0.26,0.273106
3,3_e2w_out,1.577923e+09,10.412500,1.25,1577923399,1577923376,max,10.555556,-0.31,-0.143056
4,4_e2w_out,1.577923e+09,10.829000,1.30,1577923439,1577923417,max,10.833333,-0.18,-0.004333


In [5]:
print(len(data), len(df_instant_out))

806 806


compare time error and speed error in loop detector between simulation result and calibration result

In [6]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()




,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_e2w_out,1.577923e+09,14.99,1.577923e+09,14.994000,1577923299,15.000000
1,1_e2w_out,1.577923e+09,7.08,1.577923e+09,7.080500,1577923317,6.944444
2,2_e2w_out,1.577923e+09,11.66,1.577923e+09,11.661995,1577923384,11.388889
3,3_e2w_out,1.577923e+09,10.41,1.577923e+09,10.412500,1577923399,10.555556
4,4_e2w_out,1.577923e+09,10.83,1.577923e+09,10.829000,1577923439,10.833333


In [7]:
#compare_df = compare_df.head(19)

In [8]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(0.7135607829460731)

In [9]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(0.5807692185229166)

In [10]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(0.7160553161213029)

In [11]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(0.7949614006065618)

In [12]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

time_min_error = min(compare_df['time_sim_error'].min(), compare_df['time_calib_error'].min())
time_max_error = max(compare_df['time_sim_error'].max(), compare_df['time_calib_error'].max())

-9.230000019073486 8.799999952316284
-3.440000057220459 8.52999997138977


In [13]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

speed_min_error = min(compare_df['speed_sim_error'].min(), compare_df['speed_calib_error'].min())
speed_max_error = max(compare_df['speed_sim_error'].max(), compare_df['speed_calib_error'].max())

-8.88888888888889 6.937777777777778
-6.14883544690002 8.199219632507091


In [14]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=time_min_error,  # Bin start
        end=time_max_error,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}.html")

In [15]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=speed_min_error,  # Bin start
        end=speed_max_error,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='speed error (m/s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}.html")

In [16]:
#import os
#os.environ["HORNSGATAN_HOME"]
